In [2]:
import os
directory = r'D:\github\localt20\data\processedData'  # for local
# directory = '/app/dataInHandNow/afterpreprocessed'  # for docker

import polars as pl
matches = pl.read_csv(os.path.join(directory, 'matches.csv'))
matchPlayers = pl.read_csv(os.path.join(directory, 'Matchplayers.csv')).sort('match_id')
playerStats = pl.read_csv(os.path.join(directory, 'playerStats.csv'))
matchPlayers

country,player,player_id,season,match_id
str,str,str,str,i64
"""Australia""","""AC Gilchrist""","""2b6e6dec""","""2005""",211028
"""Australia""","""ML Hayden""","""d8699ab7""","""2005""",211028
"""Australia""","""A Symonds""","""bd77eb62""","""2005""",211028
"""Australia""","""MJ Clarke""","""f842c2cf""","""2005""",211028
"""Australia""","""MEK Hussey""","""48fd7349""","""2005""",211028
…,…,…,…,…
"""Spain""","""EMC Fowler""","""4f88a917""","""2024""",1451294
"""Spain""","""Tashiba Mirza""","""2020d4fd""","""2024""",1451294
"""Spain""","""Memoona Riaz""","""3c5a7deb""","""2024""",1451294


In [3]:
matchPlayersStats = matchPlayers.join(playerStats, left_on=['player_id','season'] , right_on=['player_id','Season'], how='inner')
matchPlayersStats

country,player,player_id,season,match_id,Player,Mat,Inns,Batting Runs,SR,Ave,Country,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,Overs,Bowling_Runs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ,Dis,Ct,St,D/I
str,str,str,str,i64,str,i64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Australia""","""AC Gilchrist""","""2b6e6dec""","""2005""",211028,"""AC Gilchrist""",1,1.0,15.0,107.14,15.0,"""Australia""",8.0,16.0,2.0,70.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Australia""","""ML Hayden""","""d8699ab7""","""2005""",211028,"""ML Hayden""",1,1.0,6.0,150.0,6.0,"""Australia""",6.0,6.0,1.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
"""Australia""","""A Symonds""","""bd77eb62""","""2005""",211028,"""A Symonds""",1,1.0,0.0,0.0,0.0,"""Australia""",16.0,32.0,2.0,123.08,3.0,14.0,2.0,4.66,6.0,3.0,47.0,2.0,23.5,1.0,1.0,0.0,0.5
"""Australia""","""MJ Clarke""","""f842c2cf""","""2005""",211028,"""MJ Clarke""",1,1.0,0.0,0.0,0.0,"""Australia""",3.5,7.0,2.0,87.5,3.0,25.0,1.0,8.33,5.0,1.0,36.0,2.0,18.0,0.0,0.0,0.0,0.0
"""Australia""","""MEK Hussey""","""48fd7349""","""2005""",211028,"""MEK Hussey""",1,1.0,1.0,16.66,1.0,"""Australia""",16.0,32.0,2.0,111.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Philippines""","""ASF Sirah""","""06bcade5""","""2024""",1450767,"""ASF Sirah""",3,2.0,15.0,78.94,7.5,"""Philippines""",11.04,76.0,8.0,105.52,10.0,50.0,5.0,5.0,48.0,18.0,334.0,14.0,23.857143,6.0,6.0,0.0,0.428571
"""Philippines""","""Huzaifa Mohammed""","""65a29a30""","""2024""",1450767,"""Huzaifa Mohammed""",7,4.0,13.0,56.52,6.5,"""Philippines""",19.65,114.0,13.0,68.03,26.0,115.0,11.0,4.42,87.5,24.0,531.0,24.0,22.125,4.0,4.0,0.0,0.166667
"""Philippines""","""J Tuffin""","""8069f81f""","""2024""",1450767,"""J Tuffin""",7,7.0,95.0,89.62,31.66,"""Philippines""",31.66,95.0,7.0,89.62,20.0,117.0,8.0,5.85,20.0,8.0,117.0,6.0,19.5,0.0,0.0,0.0,0.0


In [4]:
match_id = matchPlayersStats['match_id','player_id'].with_columns(pl.lit(1).alias("one")).group_by('match_id').agg(pl.sum('one')).filter(pl.col('one') == 22)['match_id']
# matchPlayersStats['match_id','player_id'].with_columns(pl.lit(1).alias("one")).group_by('match_id').agg(pl.sum('one'))['one'].value_counts().sort(by='one')
match_id

match_id
i64
211028
211048
225263
225271
226374
…
1447492
1447496
1450753


In [5]:
matchPlayersStats = matchPlayersStats.filter(pl.col('match_id').is_in(match_id))
matchPlayersStats

country,player,player_id,season,match_id,Player,Mat,Inns,Batting Runs,SR,Ave,Country,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,Overs,Bowling_Runs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ,Dis,Ct,St,D/I
str,str,str,str,i64,str,i64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Australia""","""AC Gilchrist""","""2b6e6dec""","""2005""",211028,"""AC Gilchrist""",1,1.0,15.0,107.14,15.0,"""Australia""",8.0,16.0,2.0,70.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Australia""","""ML Hayden""","""d8699ab7""","""2005""",211028,"""ML Hayden""",1,1.0,6.0,150.0,6.0,"""Australia""",6.0,6.0,1.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
"""Australia""","""A Symonds""","""bd77eb62""","""2005""",211028,"""A Symonds""",1,1.0,0.0,0.0,0.0,"""Australia""",16.0,32.0,2.0,123.08,3.0,14.0,2.0,4.66,6.0,3.0,47.0,2.0,23.5,1.0,1.0,0.0,0.5
"""Australia""","""MJ Clarke""","""f842c2cf""","""2005""",211028,"""MJ Clarke""",1,1.0,0.0,0.0,0.0,"""Australia""",3.5,7.0,2.0,87.5,3.0,25.0,1.0,8.33,5.0,1.0,36.0,2.0,18.0,0.0,0.0,0.0,0.0
"""Australia""","""MEK Hussey""","""48fd7349""","""2005""",211028,"""MEK Hussey""",1,1.0,1.0,16.66,1.0,"""Australia""",16.0,32.0,2.0,111.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Malawi""","""GM Ntambalika""","""8c8a3f08""","""2024""",1450765,"""GM Ntambalika""",6,3.0,56.0,121.73,28.0,"""Malawi""",12.68,174.0,17.0,75.14,0.0,0.0,0.0,0.0,24.0,9.0,171.0,9.0,19.0,8.0,8.0,0.0,0.275862
"""Malawi""","""D Jakiel""","""8b7ce771""","""2024""",1450765,"""D Jakiel""",10,3.0,41.0,95.34,20.5,"""Malawi""",21.43,131.0,15.0,108.89,37.5,219.0,15.0,5.78,104.5,38.0,672.0,28.0,24.0,10.0,10.0,0.0,0.357143
"""Malawi""","""P Zuze""","""04420914""","""2024""",1450765,"""P Zuze""",9,4.0,9.0,45.0,2.25,"""Malawi""",3.5,21.0,6.0,55.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.923077


In [6]:
matchPlayersStats = matchPlayersStats.drop(['country','player','player_id','season','Player','Country'])
matchPlayersStats

match_id,Mat,Inns,Batting Runs,SR,Ave,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,Overs,Bowling_Runs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ,Dis,Ct,St,D/I
i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
211028,1,1.0,15.0,107.14,15.0,8.0,16.0,2.0,70.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211028,1,1.0,6.0,150.0,6.0,6.0,6.0,1.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
211028,1,1.0,0.0,0.0,0.0,16.0,32.0,2.0,123.08,3.0,14.0,2.0,4.66,6.0,3.0,47.0,2.0,23.5,1.0,1.0,0.0,0.5
211028,1,1.0,0.0,0.0,0.0,3.5,7.0,2.0,87.5,3.0,25.0,1.0,8.33,5.0,1.0,36.0,2.0,18.0,0.0,0.0,0.0,0.0
211028,1,1.0,1.0,16.66,1.0,16.0,32.0,2.0,111.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1450765,6,3.0,56.0,121.73,28.0,12.68,174.0,17.0,75.14,0.0,0.0,0.0,0.0,24.0,9.0,171.0,9.0,19.0,8.0,8.0,0.0,0.275862
1450765,10,3.0,41.0,95.34,20.5,21.43,131.0,15.0,108.89,37.5,219.0,15.0,5.78,104.5,38.0,672.0,28.0,24.0,10.0,10.0,0.0,0.357143
1450765,9,4.0,9.0,45.0,2.25,3.5,21.0,6.0,55.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.923077


In [7]:
matchPlayersStats.shape

(26400, 23)

In [8]:
playerStats = matchPlayersStats.to_numpy().reshape(-1,22,matchPlayersStats.width)
playerStats.shape

(600, 2, 22, 23)

In [8]:
playerStats[0][:].shape, playerStats[0][:]

((22, 23),
 array([[2.11028e+05, 1.00000e+00, 1.00000e+00, 1.50000e+01, 1.07140e+02,
         1.50000e+01, 8.00000e+00, 1.60000e+01, 2.00000e+00, 7.02400e+01,
         0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
         0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
         0.00000e+00, 0.00000e+00, 0.00000e+00],
        [2.11028e+05, 1.00000e+00, 1.00000e+00, 6.00000e+00, 1.50000e+02,
         6.00000e+00, 6.00000e+00, 6.00000e+00, 1.00000e+00, 1.50000e+02,
         0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
         0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
         1.00000e+00, 0.00000e+00, 1.00000e+00],
        [2.11028e+05, 1.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
         0.00000e+00, 1.60000e+01, 3.20000e+01, 2.00000e+00, 1.23080e+02,
         3.00000e+00, 1.40000e+01, 2.00000e+00, 4.66000e+00, 6.00000e+00,
         3.00000e+00, 4.70000e+01, 2.00000e+00, 2.35000e+01, 1.00000e+00,
   

In [9]:
import numpy as np
directory = r'D:\github\localt20\data\aftermerging'  # for local
np.save(os.path.join(directory, 'matchPlayerStats.npy'), playerStats)

In [10]:
matchPlayersStats

match_id,Mat,Inns,Batting Runs,SR,Ave,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,Overs,Bowling_Runs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ,Dis,Ct,St,D/I
i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
211028,1,1.0,15.0,107.14,15.0,8.0,16.0,2.0,70.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211028,1,1.0,6.0,150.0,6.0,6.0,6.0,1.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
211028,1,1.0,0.0,0.0,0.0,16.0,32.0,2.0,123.08,3.0,14.0,2.0,4.66,6.0,3.0,47.0,2.0,23.5,1.0,1.0,0.0,0.5
211028,1,1.0,0.0,0.0,0.0,3.5,7.0,2.0,87.5,3.0,25.0,1.0,8.33,5.0,1.0,36.0,2.0,18.0,0.0,0.0,0.0,0.0
211028,1,1.0,1.0,16.66,1.0,16.0,32.0,2.0,111.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1450765,6,3.0,56.0,121.73,28.0,12.68,174.0,17.0,75.14,0.0,0.0,0.0,0.0,24.0,9.0,171.0,9.0,19.0,8.0,8.0,0.0,0.275862
1450765,10,3.0,41.0,95.34,20.5,21.43,131.0,15.0,108.89,37.5,219.0,15.0,5.78,104.5,38.0,672.0,28.0,24.0,10.0,10.0,0.0,0.357143
1450765,9,4.0,9.0,45.0,2.25,3.5,21.0,6.0,55.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,0.0,0.923077
